In [1]:
from Hamil_search import *
from simulation import *
import numpy as np
import os
import pennylane as qml

In [2]:
def load_or_precompute(n_block):
    gx, gz = 1, 3
    if not os.path.exists(f"Hpen_{n_block}blocks.npy"):
        n_phys = 4 * n_block
        Hpen_terms = []
        for i in range(n_block):
            Hpen_terms += [(gx, f'X{4*i}*X{4*i+1}'), (gz, f'Z{4*i}*Z{4*i+1}'),
                        (gx, f'X{4*i+2}*X{4*i+3}'), (gz, f'Z{4*i+2}*Z{4*i+3}')]
        Hpen = blocks2Mat(n_phys, Hpen_terms)
        P0, HpenInverse = get_P0_HpenInverse(Hpen)
        Uenc = getU(4, n_block)
        Penc = Uenc @ Uenc.conj().T
        np.save(f'Hpen_{n_block}blocks', Hpen)
        np.save(f'HpenInverse_{n_block}blocks', HpenInverse)
        np.save(f'P0_{n_block}blocks', P0)
        np.save(f'Uenc_{n_block}blocks', Uenc)
        np.save(f'Penc_{n_block}blocks', Penc)
    
    Hpen = np.load(f'Hpen_{n_block}blocks.npy')
    HpenInverse = np.load(f'HpenInverse_{n_block}blocks.npy')
    P0 = np.load(f'P0_{n_block}blocks.npy')
    Uenc = np.load(f'Uenc_{n_block}blocks.npy')
    Penc = np.load(f'Penc_{n_block}blocks.npy')
    
    return Hpen, HpenInverse, P0, Uenc, Penc

def test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc):
    A = Henc @ HpenInverse @ Henc
    off_diag = (P0 - Penc) @ A @ Penc
    if checkSame(off_diag, np.zeros(off_diag.shape)):
        print("no leakage")
    else:
        print("yes leakage")
    Hlogi = - Uenc.conj().T @ A @ Uenc
    return qml.pauli_decompose(Hlogi)
    

def get_P0_HpenInverse(Hpen):
    """
    Make sure that the eigenvalues of Hpen are all integers
    """
    e, u = np.linalg.eigh(Hpen)

    a = np.array([1 if abs(x) < 0.5 else 0 for x in e])
    P0 = u @ (np.expand_dims(a, axis=1) * u.conj().T)

    b = np.array([1/x if abs(x) > 0.5 else 0 for x in e])
    HpenInverse = u @ (np.expand_dims(b, axis=1) * u.conj().T)

    return P0, HpenInverse


In [3]:
n_block = 2
n_phys = 4 * n_block
dim_phys = 2 ** n_phys

Hpen_terms = [(1, 'X0*X1'), (3, 'Z0*Z1'), (1, 'X2*X3'), (3, 'Z2*Z3'),
              (1, 'X4*X5'), (3, 'Z4*Z5'), (1, 'X6*X7'), (3, 'Z6*Z7')]
Hpen = blocks2Mat(n_phys, Hpen_terms)
P0, HpenInverse = get_P0_HpenInverse(Hpen)
Uenc = getU(4, n_block)
Penc = Uenc @ Uenc.conj().T

In [4]:
Henc_terms = [(1, 'Z3*X4'), (-1, 'Z3*X6')]
Henc = blocks2Mat(n_phys, Henc_terms)

test_leakage_and_get_logical_interaction(HpenInverse, P0, Penc, Uenc, Henc)

no leakage


(
    0.12499999999999994 * (I(0) @ X(1) @ I(2) @ I(3))
  + 0.12499999999999994 * (I(0) @ X(1) @ X(2) @ I(3))
)